<h1 style="text-align: center">Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto</h1>

<p><b>Step 1</b>: Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe</p>

In [1]:
!pip install beautifulsoup4
!pip install lxml

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [2]:
# import required modules
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [3]:
# assign link to Wiki page to variable
wiki_Toronto_postal_codes = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

I will use **BeautifulSoup** together with *lxml* to scrape Wikipedia page

In [4]:
# get page text and parse using BeautifulSoup
source = requests.get(wiki_Toronto_postal_codes).text
soup = BeautifulSoup(source, 'lxml')

First I will find **table** HTML tag, then all table rows **tr** and all **td** cells within row, using nested loops.
List of lists (rows data) is created, and then used to create pandas DataFrame.


In [5]:
# table with data
pcode_table = soup.find('table',{'class':'wikitable sortable'})
table_data = []
# find all table rows
for tr in pcode_table.find_all('tr'):
    row = []
    # find all cells within row
    for td in tr.find_all('td'):
        # append extracted and trimmed cell text into row data  
        row.append(td.get_text(strip=True))
    # skip adding row to table_data in case is empty (header row)
    if len(row):
        table_data.append(row)
# create data frame from list of lists
df_wiki = pd.DataFrame(data=table_data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
# filter out rows with Borough equal to 'Not assigned'
df_wiki = df_wiki[df_wiki.Borough != 'Not assigned']
df_wiki.head(15)


,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


For *Neighbourhood* with 'Not assigned' value, we need to use the *Borough* name 

In [6]:
df_wiki.loc[df_wiki['Neighbourhood'] == 'Not assigned','Neighbourhood'] = df_wiki['Borough']

I will group data frame by **PostalCode** and **Borough**, and *apply* function join more than one neighborhood for one postal code area.

In [7]:
df_grouped = df_wiki.groupby(['PostalCode', 'Borough'])['Neighbourhood'].apply(lambda neighbourhoods: ', '.join(neighbourhoods)).to_frame().reset_index()
df_grouped.head(30)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [8]:
# df_grouped shape
print('Data frame shape: ', df_grouped.shape)

Data frame shape:  (103, 3)


<p><b>Step 2:</b> Get the latitude and the longitude coordinates of each postal code in data frame<p>

<b>Note:</b> Unfortunately because unstable results, eigther using geogeocoder, Nominatim, and Nominatim with RateLimiter, I will use CSV. 

In [38]:
csv_url = 'https://cocl.us/Geospatial_data'
df_location = pd.read_csv(csv_url)
df_location.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [39]:
# rename column
df_location.rename(index=str, columns={'Postal Code': 'PostalCode'}, inplace=True)
# marge datasets on PostalCode column value
df_location = pd.merge(df_grouped, df_location, on='PostalCode')

In [41]:
print('Shape of df_location', df_location.shape)
df_location.head(10)

Shape of df_location (103, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
